In [57]:
import re
import ollama
from src.data_loading import load_data
from src.parsing import parse_options
from src.rationale_generation import generate_rationale_and_answer
from src.utils import is_rationale_correct
from src.rationalization import rationalize
from src.prompt_generation import create_prompt_examples, create_prompt_set

In [58]:
NUM_PROMPT_EXAMPLES = 10 # number of prompt examples
NUM_EXAMPLES_TO_PROCESS = 25 # number of examples to process

### Load Dataset

In [59]:
ds_train = load_data()

dataset_D = ds_train.select(range(NUM_PROMPT_EXAMPLES, len(ds_train)))
dataset_D_subset = dataset_D.select(range(NUM_EXAMPLES_TO_PROCESS))

### Create prompt examples

In [60]:
prompt_examples = create_prompt_examples(ds_train, NUM_PROMPT_EXAMPLES)

prompt_examples

[{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
  'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'],
  'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
  'answer': 'E'},
 {'question': 'In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?',
  'options': ['A)4 and 1', 'B)1 and 5', 'C)5 and 1', 'D)3 and 5', 'E)5 and 3'],
  'rationale': 'Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.\nThus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1\nAnswer: C',
  'answer': 'C'},
 {'question': 'For all numbers p and

### Create Prompt Sets

In [61]:
prompt_set = create_prompt_set(ds_train, NUM_PROMPT_EXAMPLES)
prompt_set

"Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?\nOptions:\nA)21\nB)21.5\nC)22\nD)22.5\nE)23\nAnswer Explanation: If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.\nAnswer: E\nPlease respond with only the letter corresponding to the correct answer (A, B, C, D, or E).\n###\nQuestion: In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?\nOptions:\nA)4 and 1\nB)1 and 5\nC)5 and 1\nD)3 and 5\nE)5 and 3\nAnswer Explanation: Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.\nThus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1\nAnswer: C

### Initialize lists to hold correct and incorrect pairs

In [62]:
correct_pairs = []
incorrect_pairs = []

### Iterate over each example in the subset

In [63]:
for idx, example in enumerate(dataset_D_subset):
    question = example['question']
    options_list = [opt.strip() for opt in example['options']]
    correct_option = example['correct'].strip().upper()
    
    # Parse options into a dictionary
    options_dict = parse_options(options_list)
    
    # Extract the correct answer text
    correct_answer_text = options_dict.get(correct_option, None)
    
    if correct_answer_text is None:
        print(f"Warning: Correct answer not found for index {idx}. Skipping this example.")
        continue  # Skip if the correct answer is missing
    
    # Generate rationale and answer using the updated function
    generated_rationale, generated_answer_text = generate_rationale_and_answer(question, options_dict, prompt_set)
    
    # Initialize generated_answer_letter
    generated_answer_letter = None
    
    # Map the generated answer text to the option letter
    if generated_answer_text:
        generated_answer_letter = generated_answer_text.strip().upper()
    
    match = re.search(r'And the answer is:\s*([A-E])', generated_rationale, re.IGNORECASE)
    if match:
        generated_answer_letter = match.group(1).upper()
    
    # Categorize the example based on the accuracy of the generated answer
    if (generated_answer_letter == correct_option) and is_rationale_correct(generated_rationale, correct_option, question):
        # Correct answer
        correct_pairs.append({
            'question': question,
            'options': options_dict,
            'rationale': generated_rationale,
            'answer': correct_option
        })
        print('Correct:', {
            'question': question,
            'options': options_dict,
            'rationale': generated_rationale,
            'generated_answer': generated_answer_letter,
            'correct_answer': correct_option
        })
    else:
        # Incorrect answer
        incorrect_pairs.append({
            'question': question,
            'options': options_dict,
            'rationale': generated_rationale,
            'generated_answer': generated_answer_letter,
            'correct_answer': correct_option
        })
        print("Incorrect:",  {
            'question': question,
            'options': options_dict,
            'rationale': generated_rationale,
            'generated_answer': generated_answer_letter,
            'correct_answer': correct_option
        })
    
    # Print progress every example
    print(f"Processed {idx + 1} questions.\n")

Correct: {'question': 'If Tim had lunch at $50 and he gave 20% tip, how much did he spend?', 'options': {'A': '$60.00', 'B': '$35.42', 'C': '$60.60', 'D': '$21.56', 'E': '$78.45'}, 'rationale': "To find out how much Tim spent on lunch, we need to calculate the total cost including the tip.\n\nFirst, let's calculate the amount of the tip:\n20% of $50 = 0.2 x $50 = $10\n\nNow, let's add the tip to the original price of lunch:\n$50 (lunch) + $10 (tip) = $60\n\nTherefore, Tim spent a total of $60 on lunch.\n\nAnd the answer is: A)", 'generated_answer': 'A', 'correct_answer': 'A'}
Processed 1 questions.

Incorrect: {'question': 'Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest.If the rate of interest is increased by 4% ,What amount will Rs. 825 become in 3 years ?', 'options': {'A': 'Rs. 1020.80', 'B': 'Rs. 1025', 'C': 'Rs. 1055', 'D': 'Data inadequate', 'E': 'None of these'}, 'rationale': "To solve this problem, we'll first find the rate at which the interest is calc

In [22]:
correct_pairs

[{'question': 'If Tim had lunch at $50 and he gave 20% tip, how much did he spend?',
  'options': {'A': '$60.00',
   'B': '$35.42',
   'C': '$60.60',
   'D': '$21.56',
   'E': '$78.45'},
  'rationale': "To find out how much Tim spent in total, we need to calculate the tip first and then add it to the cost of lunch.\n\nThe cost of lunch is $50.\nThe tip is 20% of the cost of lunch. To find 20% of a number, we multiply the number by 0.2 (which is equivalent to multiplying by 1/5).\n\nSo, the tip is: $50 x 0.2 = $10\n\nNow, let's add the tip to the cost of lunch:\n$50 + $10 = $60\n\nTherefore, Tim spent a total of $60 on lunch.\n\nAnd the answer is: A)",
  'answer': 'A'},
 {'question': 'q is a positive integer and multiple of 2; p = 4^q, what is the remainder when p is divided by 10?',
  'options': {'A': '10',
   'B': '6',
   'C': '4',
   'D': '0',
   'E': 'It Cannot Be Determined'},
  'rationale': "To find the remainder when p (which equals 4^q) is divided by 10, we need to examine the p

In [23]:
incorrect_pairs

[{'question': 'Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest.If the rate of interest is increased by 4% ,What amount will Rs. 825 become in 3 years ?',
  'options': {'A': 'Rs. 1020.80',
   'B': 'Rs. 1025',
   'C': 'Rs. 1055',
   'D': 'Data inadequate',
   'E': 'None of these'},
  'rationale': "Let's break down the problem step by step.\n\nFirst, we need to find the rate of interest (R) at which Rs. 825 becomes Rs. 956 in 3 years. We can use the simple interest formula:\n\nSimple Interest = (Principal × Rate × Time) / 100\n\nwhere Principal = Rs. 825, and Simple Interest = Rs. 956 - Rs. 825 = Rs. 131.\n\nWe are given that this happens over a period of 3 years. So,\n\nRs. 131 = (Rs. 825 × R × 3) / 100\n\nTo find the rate of interest (R), we can rearrange the formula:\n\nR = (Rs. 131 × 100) / (Rs. 825 × 3)\n\nR = Rs. 13,100 / Rs. 2,475\n\nR ≈ 5.29%\n\nNow that we have found the original rate of interest (R), let's say it increases by 4% to become R + 0.04.\n\nThe

Testing it out. 

In [ ]:
# # Test with a specific example
# test_example = dataset_D_subset[2]  # Adjust the index as needed
# question = test_example['question']
# options_list = [opt.strip() for opt in test_example['options']]
# correct_option = test_example['correct'].strip().upper()

# # Parse options into a dictionary
# options_dict = parse_options(options_list)

# # Extract the correct answer
# correct_answer = options_dict.get(correct_option, None)

# print("Question:", question)
# print("Options:", options_dict)
# print("Correct Option:", correct_option)
# print("Correct Answer:", correct_answer)

# # Generate rationale and answer
# generated_rationale, generated_answer_text = generate_rationale_and_answer(question, options_dict, prompt_set)

# print("\nGenerated Rationale:")
# print(generated_rationale)
# print("\nGenerated Answer:", generated_answer_text)
# print("Correct Answer:", correct_answer)


In [64]:
total = len(correct_pairs) + len(incorrect_pairs)
accuracy = len(correct_pairs) / total * 100
print(f"Total questions processed: {total}")
print(f"Correct answers: {len(correct_pairs)}")
print(f"Incorrect answers: {len(incorrect_pairs)}")
print(f"Accuracy: {accuracy:.2f}%")

Total questions processed: 25
Correct answers: 15
Incorrect answers: 10
Accuracy: 60.00%


In [26]:
new_correct_pairs = []

# Process the incorrect answers
for pair in incorrect_pairs:
    question = pair['question']
    options = pair['options']
    correct_answer = pair['correct_answer']

    # Generate the rationale with the correct answer as a hint
    generated_rationale = rationalize(question, options, correct_answer, prompt_set)

    # Add the rationalized example to correct_pairs
    new_correct_pairs.append({
        'question': question,
        'options': options,
        'rationale': generated_rationale,
        'answer': correct_answer
    })

    print({
        'question': question,
        'options': options,
        'rationale': generated_rationale,
        'answer': correct_answer
    })

{'question': 'Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest.If the rate of interest is increased by 4% ,What amount will Rs. 825 become in 3 years ?', 'options': {'A': 'Rs. 1020.80', 'B': 'Rs. 1025', 'C': 'Rs. 1055', 'D': 'Data inadequate', 'E': 'None of these'}, 'rationale': "Let's break down the problem step by step.\n\nGiven:\n\n1. Principal (P) = Rs. 825\n2. Amount after 3 years (A) = Rs. 956\n\nWe need to find the rate of interest (R) at which P becomes A in 3 years. We'll use the formula for simple interest:\n\nSimple Interest (SI) = (P × R × T)/100\n\nwhere T is the time period, which is 3 years here.\n\nFirst, let's calculate the SI for the given scenario:\n\nA = P + SI\n956 = 825 + (SI)\nSI = 131\n\nNow, we can use the formula to find the rate of interest (R):\n\nSI = (P × R × T)/100\n131 = (825 × R × 3)/100\n131 = 24.75R\nR = 131/24.75 ≈ 5.29%\n\nThis is the original rate of interest.\n\nNow, let's consider what happens when the rate of interest is i

### Converting our corrected pairs into a structure the Llama3.1 model will understand for fine tuning.

In [47]:
from datasets import Dataset, DatasetDict


def convert_correct_pairs_to_conversations(correct_pairs):
    """
    Converts correct_pairs into the desired conversations format.

    Parameters:
        correct_pairs (list): List of dictionaries containing question, options, correct_answer, and rationale.

    Returns:
        list: A list of conversations formatted for fine-tuning.
    """
    conversations = []
    
    for pair in correct_pairs:
        question = pair['question']
        options = pair['options']
        correct_answer = pair['answer']
        rationale = pair['rationale']
        
        # Construct options text
        options_text = '\n'.join([f"{key}: {value}" for key, value in options.items()])
        
        # Human prompt
        human_value = (
            f"Question: {question}\n"
            f"Options:\n{options_text}\n\n"
            f"Please provide the correct answer and a detailed rationale explaining why it's correct."
        )
        
        # GPT response
        gpt_value = (
            f"{correct_answer}\nRationale: {rationale}"
        )
        
        # Append the conversation as a separate list
        conversation = [
            { "from": "human", "value": human_value },
            { "from": "gpt", "value": gpt_value }
        ]
        
        conversations.append(conversation)
    
    return conversations

conversations = convert_correct_pairs_to_conversations(correct_pairs)

# Create a Hugging Face Dataset
dataset = Dataset.from_dict({"conversations": conversations})

# Create a DatasetDict with only the 'train' split
dataset_dict = DatasetDict({
    "train": dataset
})

# Optionally, shuffle the dataset
dataset_dict["train"] = dataset_dict["train"].shuffle(seed=42)

# Verify the structure
print(dataset_dict)
print(dataset_dict["train"][0])  # Print the first conversation

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 5
    })
})
{'conversations': [{'from': 'human', 'value': "Question: Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest.If the rate of interest is increased by 4% ,What amount will Rs. 825 become in 3 years ?\nOptions:\nA: Rs. 1020.80\nB: Rs. 1025\nC: Rs. 1055\nD: Data inadequate\nE: None of these\n\nPlease provide the correct answer and a detailed rationale explaining why it's correct."}, {'from': 'gpt', 'value': "C\nRationale: To solve this problem, we need to find the original rate of interest and then calculate the new amount.\n\n**Step 1: Find the original rate of interest**\n\nWe know that Rs. 825 becomes Rs. 956 in 3 years at a certain rate of simple interest. We can use the formula for simple interest:\n\nSimple Interest (SI) = (Principal × Rate × Time) / 100\n\nHere, Principal = Rs. 825, Time = 3 years, and we need to find the Rate.\n\nFirst, let's calculate the Simpl

### Save dataset to JSONL format

In [48]:
import json

def save_dataset_to_jsonl(dataset_dict, file_path='finetuning_data.jsonl'):
    """
    Saves the conversations in dataset_dict to a JSONL file.
    
    Parameters:
        dataset_dict (DatasetDict): The dataset to save.
        file_path (str): The path to the output JSONL file.
        
    Returns:
        None
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for conversation in dataset_dict["train"]["conversations"]:
            json_line = json.dumps(conversation)
            f.write(json_line + '\n')
    print(f"Dataset successfully saved to {file_path}")

# Usage
save_dataset_to_jsonl(dataset_dict, 'finetuning_data.jsonl')

Dataset successfully saved to finetuning_data.jsonl


### Load dataset from JSONL (for appending new data)

In [ ]:
def load_dataset_from_jsonl(file_path='finetuning_data.jsonl', validation_split=0.1):
    """
    Loads conversations from a JSONL file and splits them into train and validation sets.

    Parameters:
        file_path (str): Path to the JSONL file.
        validation_split (float): Proportion of data to use for validation.

    Returns:
        DatasetDict: A DatasetDict with 'train' and 'validation' splits.
    """
    dataset = Dataset.from_json(file_path)
    dataset = dataset.train_test_split(test_size=validation_split, seed=42)
    dataset_dict = DatasetDict({
        "train": dataset["train"],
        "validation": dataset["test"]
    })
    print(f"Loaded dataset with {len(dataset_dict['train'])} training and {len(dataset_dict['validation'])} validation conversations.")
    return dataset_dict

### Appending New Data

In [50]:
new_conversations = convert_correct_pairs_to_conversations(new_correct_pairs)

def append_conversations_to_jsonl(conversations, file_path='finetuning_data.jsonl'):
    """
    Appends a list of conversations to an existing JSONL file.

    Parameters:
        conversations (list): List of conversations to append.
        file_path (str): Path to the existing JSONL file.

    Returns:
        None
    """
    with open(file_path, 'a', encoding='utf-8') as f:
        for convo in conversations:
            json_line = json.dumps(convo)
            f.write(json_line + '\n')
    print(f"Successfully appended {len(conversations)} conversations to {file_path}")
    
# Usage
append_conversations_to_jsonl(new_conversations, 'finetuning_data.jsonl')

Successfully appended 5 conversations to finetuning_data.jsonl
